# LFPCA through all conditions of anesthetized monkey

to run the monkey data through the analysis pipeline <br>
eyes open, eyes closed, and anesthesized <br>
write the code such that running through the notebook ONCE will reproduce all the results for all 3 datasets

visualize the results and compare the different conditions

In [63]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/Lauren/Voytek/spectralCV')
sys.path.append('/Users/rdgao/Documents/code/research/spectralCV')
sys.path.append('/Users/rdgao/Documents/code/research/neurodsp/')
sys.path.append('/Users/rdgao/Documents/code/research/fooof/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
%matplotlib inline

# imports
from scv_funcs import access_nt as asc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import neurodsp as ndsp
from fooof import FOOOFGroup
from scv_funcs import lfpca

In [65]:
#data_path ="/Users/Lauren/Data/NeuroTycho/anesthesia/20120730PF_Anesthesia+and+Sleep_Chibi_Toru+Yanagawa_mat_ECoG128/Session%d/"
data_path ="/Users/rdgao/Documents/data/NeuroTycho/Propofol/20120730PF_Anesthesia+and+Sleep_Chibi_Toru+Yanagawa_mat_ECoG128/Session%d/"
saveout_path = "../results/nt/"

In [66]:
# fooof wrapper
def fooof_lfpca(lfpca, background_mode, freq_range=[3,100]):
    fg = FOOOFGroup(background_mode=background_mode, verbose=False)
    fg.fit(lfpca.f_axis, lfpca.psd, freq_range)
    slopes = np.array([r.background_params[-1] for r in fg.get_results()])
    if background_mode is 'knee':
        knees = np.array([r.background_params[1] for r in fg.get_results()])
    else:
        knees = np.ones_like(slopes)
    r2s = np.array([r.r_squared for r in fg.get_results()])
    return np.array([slopes, knees, r2s]).T

In [109]:
# example of using LFPCA
fs = 1000
analysis_param = {'nperseg': 1000,
                 'noverlap': 0,
                 'spg_outlierpct': 2.,
                 'max_freq':200}
session_indices = [(1,0,1), (1,2,3), (2,1,2), (2,3,4), (3,0,1)]
session_labels = ['EyesOpen', 'EyesClosed', 'Anes', 'RecEyesClosed', 'RecEyesOpen']
fooof_conds = [('fixed',[3,100]),('knee',[3,100]),('fixed',[40,70])]


for i in range(2,len(session_indices)):
    session_ind = session_indices[i]
    session = session_ind[0]
    start_ind = session_ind[1]
    end_ind = session_ind[2]
    chan = range(1,129)
    #chan = [1,2]
    
    # with specified indices
    indices = asc.get_cond(data_path, session, start_ind, end_ind)
    data = asc.get_ECoG(data_path, session, chan, indices)

    # initialize object with data and parameters
    nt_lfpca = lfpca.LFPCA(analysis_param)
    nt_lfpca.populate_ts_data(data, fs)

    # compute all spectral attributes
    nt_lfpca.compute_all_spectral()

    # compute KS-test fit against null exponential hypothesis
    nt_lfpca.compute_KS_expfit() # this takes a few minutes
    # save out
    nt_lfpca.save_spec_vars('../results/nt/nt_lfpca%i_'%i+session_labels[i])
    
    nt_fooof = np.zeros((nt_lfpca.numchan, 3, len(fooof_conds)))
    for fc in range(len(fooof_conds)):
        nt_fooof[:,:,fc] = fooof_lfpca(nt_lfpca, fooof_conds[fc][0], freq_range=fooof_conds[fc][1])
    
    np.savez('../results/nt/nt_fooof%i_'%i+session_labels[i], 
             fooof_res=nt_fooof, 
             fooof_bg_param=np.array([fooof_conds[fc][0] for fc in range(len(fooof_conds))]),
             fooof_freq_range=np.array([fooof_conds[fc][1] for fc in range(len(fooof_conds))])
            )

Anesthetized-Start Anesthetized-End
Handling run-time error: Did not fully consume compressed contents of an miCOMPRESSED element. This can indicate that the .mat file is corrupted.
Channel 63 is filled in with noise.


/Users/rdgao/Documents/code/research/fooof/fooof/core/funcs.py:65: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)
/Users/rdgao/Documents/code/research/fooof/fooof/core/funcs.py:65: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)


RecoveryEyesClosed-Start RecoveryEyesClosed-End
Handling run-time error: Did not fully consume compressed contents of an miCOMPRESSED element. This can indicate that the .mat file is corrupted.
Channel 63 is filled in with noise.
RecoveryEyesOpened-Start RecoveryEyesOpened-End


In [24]:
# fooof lfpca
def fooof_monkey(nt_lfpca_monkey):
    all_bg_params = []
    all_r2 = []
    for i in range(5):
        linear_bg_params, linear_r2 = fooof_lfpca(nt_lfpca_monkey[i],'fixed')
        lz_bg_params, lz_r2 = fooof_lfpca(nt_lfpca_monkey[i],'knee')
        noise_bg_params, noise_r2 = fooof_lfpca(nt_lfpca_monkey[i],'fixed', freq_range=[30,70])
        bg_params = (linear_bg_params, lz_bg_params, noise_bg_params)
        r2 = (linear_r2, lz_r2, noise_r2)
        all_bg_params.append(bg_params)
        all_r2.append(r2)
    return all_bg_params, all_r2

In [25]:
all_bg_params, all_r2 = fooof_monkey(nt_lfpca_monkey=nt_lfpca_monkey)


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 1.00 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is freq-res)
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 1.00 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is freq-res)
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 1.00 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is freq-res)
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resoluti